In [32]:
# Given a credit card csv file in Nationwide (UK) format,
# generate a csv format acceptable to Homebank.
# Nationwide excludes location data from its ofx output,
# hence no ofx conversion here.
# Homebank - it turns out that the [info] and [tag] fields are never 
# exported to the QIF format. But they can be exported
# as the native .XHB format, so may have some value.
# Given the total absence of good practice here, there are 
# many todos.

# include these in requirements.txt:
import pandas as pd
import pprint
from datetime import datetime

# Note the encoding as delivered by Nationwide
nw_encoding = 'cp1252'
in_file = r"ds2023.csv"
out_file = r"output.csv"
input_df = pd.read_csv(in_file, skiprows=4, encoding=nw_encoding)


# Process data and create the output DataFrame
output_df = pd.DataFrame({
    "Date": input_df["Date"],
    "payment type": 1,  # placeholder
    "info": "info 1",  # placeholder
    "Transactions": input_df["Transactions"],
    "Location": input_df["Location"],
    "Paid out": input_df["Paid out"].astype(str).str.replace("£", ""),
    "Paid in": input_df["Paid in"].astype(str).str.replace("£", ""),
    "category": "category 1",  # placeholder
    "Tags": "tag1 tag2"  # placeholder
})

# Do the more complex stuff in a second pass
# When looking through Homebank transactions, it is useful to know where
# one statement ends, and another starts
output_df.loc[0,'info'] = 'First transaction in statement'
output_df.loc[output_df.index[-1],'info'] = 'Last transaction in statement'

output_df.fillna(0,inplace=True)
output_df["Paid out"] = output_df["Paid out"].astype(float)
output_df["Paid in"] = output_df["Paid in"].astype(float)
output_df["Paid"] = output_df.apply(lambda row: (row["Paid out"] * -1) if row["Paid out"] > 0.0 else row["Paid in"], axis = 1)
output_df["Date"] = pd.to_datetime(output_df["Date"], format="%d %b %Y")

# Finally in a form that HomeBank can ingest:
columns_to_delete = ["Paid out", "Paid in"] 
output_df.drop(columns=columns_to_delete, inplace=True)
desired_order = [0,1,2,3,4,7,5,6]
output_df = output_df.iloc[:, desired_order]

pprint.pprint(output_df) # debug

output_df.to_csv(out_file, sep=";", index=False, header=False)



        Date  payment type                            info  \
0 2023-05-12             1  First transaction in statement   
1 2023-05-14             1                          info 1   
2 2023-05-16             1                          info 1   
3 2023-05-17             1                          info 1   
4 2023-05-18             1                          info 1   
5 2023-05-18             1                          info 1   
6 2023-05-18             1                          info 1   
7 2023-05-23             1                          info 1   
8 2023-05-23             1   Last transaction in statement   

                    Transactions    Location   Paid    category       Tags  
0      BOOTS OPTICIANS  ApplePay     EXMOUTH  -8.98  category 1  tag1 tag2  
1                LIDL GB EXMOUTH     EXMOUTH -14.22  category 1  tag1 tag2  
2          eBay O*13-10065-61792      LONDON  -2.98  category 1  tag1 tag2  
3    Exmouth Cycles  Contactless     Exmouth -40.00  category 1  tag1 t